## Parse JSON

In [1]:
import json
import numpy as np
with open('./data/Twitter/tweets_DM.json', 'r') as f:
    text = f.read()
#tt = text.splitlines()[0]
#json.loads(tt)
data = [json.loads(line) for line in text.splitlines()]

# returns JSON object as 
# a dictionary
#data = json.load(f)
#print(data)

In [2]:
sample_data = data[0]
sample_data

{'_score': 391,
 '_index': 'hashtag_tweets',
 '_source': {'tweet': {'hashtags': ['Snapchat'],
   'tweet_id': '0x376b20',
   'text': 'People who post "add me on #Snapchat" must be dehydrated. Cuz man.... that\'s <LH>'}},
 '_crawldate': '2015-05-23 11:42:47',
 '_type': 'tweets'}

In [24]:
import pandas as pd
df = pd.DataFrame([i['_source']['tweet'] for i in data])
df_identification = pd.read_csv('./data/Twitter/data_identification.csv')
df_emotion = pd.read_csv('./data/Twitter/emotion.csv')

In [25]:
df = df.merge(df_identification, how='left' ,on='tweet_id')
df = df.merge(df_emotion, how='left' ,on='tweet_id')
df.head()


,hashtags,tweet_id,text,identification,emotion
0,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation
1,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test,NaN
3,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test,NaN


In [34]:
train_df = df.loc[df['identification']=='train', :]
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(train_df, test_size=0.2)

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

# build analyzers (bag-of-words)
BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
BOW_500.fit(train_df['text'])

#train_data_BOW_features_500 = BOW_500.transform(train_df['text'])

## check dimension
#train_data_BOW_features_500.shape

c:\Users\Kristofferson\anaconda3\envs\DMlab2\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(1164450, 500)

In [36]:
import keras

# standardize name (X, y) 
X_train = BOW_500.transform(train_df['text'])
y_train = train_df['emotion']

X_test = BOW_500.transform(test_df['text'])
y_test = test_df['emotion']

## check dimension is a good habbit 
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (1164450, 500)
y_train.shape:  (1164450,)
X_test.shape:  (291113, 500)
y_test.shape:  (291113,)


In [37]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)


check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 130774             fear
1026083           trust
426705              joy
1619145    anticipation
Name: emotion, dtype: object

y_train.shape:  (1164450,)
y_test.shape:  (291113,)


C:\Users\Kristofferson\AppData\Local\Temp\ipykernel_45700\393062722.py:9: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  print('y_train[0:4]:\n', y_train[0:4])




## After convert
y_train[0:4]:
 [[0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]]

y_train.shape:  (1164450, 8)
y_test.shape:  (291113, 8)


In [38]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  500
output_shape:  8


In [39]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500)]             0         
                                                                 
 dense (Dense)               (None, 64)                32064     
                                                                 
 re_lu (ReLU)                (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 re_lu_1 (ReLU)              (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
 softmax (Softmax)           (None, 8)                 0     

In [40]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 25
batch_size = 32

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_test, y_test))
print('training finish')
# interrupted due to overfitting

Epoch 1/25
36390/36390 [==============================] - 83s 2ms/step - loss: 1.4195 - accuracy: 0.4819 - val_loss: 1.3866 - val_accuracy: 0.4946
Epoch 2/25
36390/36390 [==============================] - 80s 2ms/step - loss: 1.3783 - accuracy: 0.4966 - val_loss: 1.3784 - val_accuracy: 0.4985
Epoch 3/25
36390/36390 [==============================] - 85s 2ms/step - loss: 1.3675 - accuracy: 0.5009 - val_loss: 1.3726 - val_accuracy: 0.4999
Epoch 4/25
36390/36390 [==============================] - 83s 2ms/step - loss: 1.3606 - accuracy: 0.5038 - val_loss: 1.3696 - val_accuracy: 0.5009
Epoch 5/25
36390/36390 [==============================] - 84s 2ms/step - loss: 1.3565 - accuracy: 0.5053 - val_loss: 1.3706 - val_accuracy: 0.5012
Epoch 6/25
36390/36390 [==============================] - 99s 3ms/step - loss: 1.3534 - accuracy: 0.5063 - val_loss: 1.3704 - val_accuracy: 0.5026
Epoch 7/25
36390/36390 [==============================] - 105s 3ms/step - loss: 1.3510 - accuracy: 0.5072 - val_loss: 

KeyboardInterrupt: 

In [41]:
test_df = df.loc[df['identification']=='test', :]
X_test = BOW_500.transform(test_df['text'])

In [44]:

pred_result = model.predict(X_test, batch_size=128)
pred_result[:5]
pred_result = label_decode(label_encoder, pred_result)
pred_result[:5]

3219/3219 [==============================] - 7s 2ms/step


array(['anticipation', 'trust', 'sadness', 'joy', 'anticipation'],
      dtype=object)

In [45]:
test_df['emotion'] = pred_result

C:\Users\Kristofferson\AppData\Local\Temp\ipykernel_45700\3331844584.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['emotion'] = pred_result


In [52]:
output = test_df[['tweet_id', 'emotion']]
output.rename(columns={'tweet_id': 'id'}, inplace=True)
output.head()

C:\Users\Kristofferson\AppData\Local\Temp\ipykernel_45700\3483519020.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output.rename(columns={'tweet_id': 'id'}, inplace=True)


,id,emotion
2,0x28b412,anticipation
4,0x2de201,trust
9,0x218443,sadness
30,0x2939d5,joy
33,0x26289a,anticipation


In [53]:
output.to_csv('output.csv', index=False)

![](kaggle.png)